In [1]:
!pip install PyPDF2

In [2]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz --no-deps

     |████████████████████████████████| 12.0 MB 3.1 MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047106 sha256=c4c76e086ab839240b2c5823aa7abb8216fcde8f963ae856e15295e80a3ca004
  Stored in directory: /Users/shounakmondal/Library/Caches/pip/wheels/b7/0d/f0/7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm


In [3]:
!pip install pyenchant

In [4]:
import spacy
import PyPDF2
import os

In [5]:
# spacy english model (large)
nlp = spacy.load('en_core_web_lg')

In [6]:
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io

# method for reading a pdf file
def readPdfFile(filename, folder_name):
    
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open('./docs/doc.pdf', 'rb') as fh:

        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    # close open handles
    converter.close()
    fake_file_handle.close()


    return text

In [7]:
# customer sentence segmenter for creating spacy document object
def setCustomBoundaries(doc):
    # traversing through tokens in document object
    for token in doc[:-1]:
        if token.text == ';':
            doc[token.i + 1].is_sent_start = True
        if token.text == ".":
            doc[token.i + 1].is_sent_start = False
    return doc


In [8]:
# create spacy document object from pdf text
def getSpacyDocument(pdf_text, nlp):
    main_doc = nlp(pdf_text)  # create spacy document object

    return main_doc

In [9]:
# adding setCusotmeBoundaries to the pipeline
nlp.add_pipe(setCustomBoundaries, before='parser')

In [10]:
from scipy import spatial

import enchant
d = enchant.Dict("en_US")

from spacy.tokens import Doc

# convert keywords to vector
def createKeywordsVectors(keyword, nlp):
    print("Searching document for word: ",keyword)
    doc = nlp(keyword)  # convert to document object
    #doc = Doc(nlp.vocab, words = keyword)  # convert to document object
    #print("doc",doc.vector)
    return doc.vector


# method to find cosine similarity
def cosineSimilarity(vect1, vect2):
    # return cosine distance
    return 1 - spatial.distance.cosine(vect1, vect2)


# method to find similar words
def getSimilarWords(keyword, nlp):
    similarity_list = []

    keyword_vector = createKeywordsVectors(keyword, nlp)

    for tokens in nlp.vocab:
        if (tokens.has_vector):
            if (tokens.is_lower):
                if (tokens.is_alpha):
                    similarity_list.append((tokens, cosineSimilarity(keyword_vector, tokens.vector)))

    similarity_list = sorted(similarity_list, key=lambda item: -item[1])
    similarity_list = similarity_list[:30]
    top_similar_words = [item[0].text for item in similarity_list]
    top_similar_words = top_similar_words[:3]
    top_similar_words.append(keyword)
    #print("top_similar_words:1",top_similar_words)
#     for token in Doc(nlp.vocab, words = keyword):
#         top_similar_words.insert(0, token.lemma_)

#     for words in top_similar_words:
#         if words.endswith("s"):
#             top_similar_words.append(words[0:len(words)-1])

#     top_similar_words = list(set(top_similar_words))
    #print("top_similar_words:",top_similar_words)
    #top_similar_words = [words for words in top_similar_words if enchant_dict.check(words) == True]
    top_similar_words = [words for words in top_similar_words if d.check(words) == True]
    
    top_similar_words = [words for words in top_similar_words]
    similarity_list.clear()
    return ", ".join(top_similar_words)


In [48]:
import gensim,nltk
from nltk.data import find
nltk.download("word2vec_sample")
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

[nltk_data] Downloading package word2vec_sample to
[nltk_data]     /Users/shounakmondal/nltk_data...
[nltk_data]   Package word2vec_sample is already up-to-date!


In [85]:
#keywords = ["safety","guidelines","manufacturers"]
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
similar_keywords_list = []
similar_keywords = ""
keyword = "justice"
#similar_keywords = getSimilarWords(keyword, nlp)
for word_tuple in word2vec_model.similar_by_word(keyword)[:5]:
    similar_keywords_list.append(word_tuple[0])
    
similar_keywords = " ".join(similar_keywords_list)
print("Similar words are:",similar_keywords)


Similar words are: judicial Justice equality injustice judiciary


In [90]:
from spacy.matcher import PhraseMatcher
from scipy import spatial

# method for searching keyword from the text
def search_for_keyword(keyword, doc_obj, nlp):
    phrase_matcher = PhraseMatcher(nlp.vocab)
    li = list(keyword.split(" ")) 
    print(li)
    patterns = [nlp.make_doc(text) for text in li]
    phrase_matcher.add("TerminologyList", patterns)

    matched_items = phrase_matcher(doc_obj)
    print(matched_items)
    matched_text = []
    for match_id, start, end in matched_items:
        text = nlp.vocab.strings[match_id]
        span = doc_obj[start: end]
        matched_text.append(span.sent.text)
    
    for txt in matched_text:
        print("Found Match ####")
        print(txt)
    

    

In [91]:
pdf = readPdfFile("testdoc.pdf","docs")

In [92]:
doc_obj = getSpacyDocument(pdf,nlp)

In [93]:
print("Searching for...",similar_keywords,"\n")
search_for_keyword(similar_keywords,doc_obj, nlp)

Searching for... judicial Justice equality injustice judiciary 

['judicial', 'Justice', 'equality', 'injustice', 'judiciary']
[(3766102292120407359, 43, 44), (3766102292120407359, 59, 60), (3766102292120407359, 122, 123), (3766102292120407359, 134, 135), (3766102292120407359, 225, 226), (3766102292120407359, 235, 236), (3766102292120407359, 326, 327), (3766102292120407359, 400, 401), (3766102292120407359, 432, 433), (3766102292120407359, 437, 438), (3766102292120407359, 799, 800), (3766102292120407359, 816, 817), (3766102292120407359, 843, 844), (3766102292120407359, 887, 888), (3766102292120407359, 926, 927), (3766102292120407359, 986, 987), (3766102292120407359, 998, 999), (3766102292120407359, 1041, 1042), (3766102292120407359, 1051, 1052), (3766102292120407359, 1105, 1106), (3766102292120407359, 1134, 1135), (3766102292120407359, 1503, 1504), (3766102292120407359, 1554, 1555), (3766102292120407359, 1562, 1563), (3766102292120407359, 1576, 1577), (3766102292120407359, 1658, 1659), 